In [1]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './data'

논문을 벡터 db에 넣기

In [2]:
# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

### 인적정보 가져오기

### 프롬프트

모델 선언

In [3]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = '''
너는 사회복지사의 업무를 도와주기 위한 챗봇이다. \\
사회복지 업무와 관련된 메뉴얼과 가이드북을 읽어서 사용자의 질문에 답변할 수 있도록 학습되었다. \\
너는 주어진 업무를 아주 잘 한다. \\
Answer the question based only on the following context:
{context}

Question: {question}

'''

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(api_key=OPENAI_API_KEY,model='gpt-4o', temperature=0)
# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  """
    사회복지시설에서의 업무를 수행하다가 발생한 사고들에 대해 어떻게 대처해야 할까요?
"""    # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(rag_chain.invoke(input_message))


> 사회복지시설에서의 업무를 수행하다가 발생한 사고들에 대해 다음과 같이 대처해야 합니다:
> 
> 1. **즉각적인 안전조치**:
>    - 사고 발생 시 즉각적인 안전조치를 취합니다. 예를 들어, 부상자가 발생한 경우 응급처치를 실시하고, 필요 시 응급의료 및 소방서에 연락합니다.
> 
> 2. **초기 대응 및 긴급 조치**:
>    - 폭력 사건이 발생하면 폭력피해 예방·관리 담당자가 주도하여 초기 대응을 합니다. 긴급한 조치가 이루어졌고 초기대응을 마친 후에는 사고 수습을 시행합니다.
> 
> 3. **침착한 대응**:
>    - 위급한 상황일 때 빠른 판단이 중요하지만, 침착한 대응이 제2의 피해를 최소화할 수 있으므로 침착하게 진행합니다.
> 
> 4. **사고 보고서 작성**:
>    - 사고보고서는 작성요령에 따라 사고 당사자가 문서화하는 것이 중요합니다.
> 
> 5. **사고 원인 및 대책 공유**:
>    - 동료 사회복지시설 종사자와 사고의 요인과 대책을 공유하여 재발하지 않도록 합니다. 사고 당사자에 대해서도 동료들이 심리적 지지를 해주는 것이 중요합니다.
> 
> 6. **의료비 보장**:
>    - 사회복지시설 종사자가 폭력으로 인해 의료비가 발생한 경우, 사회복지공제회의 ‘정부지원 단체 상해 공제’를 통해 보장받을 수 있도록 합니다.
> 
> 7. **정기적 점검 및 예방훈련**:
>    - 시설 내외 안전에 관한 정기적 점검을 실시하고, 사고 재발 및 폭력피해 예방을 위한 교육훈련 계획을 수립하여 지속적으로 훈련을 수행합니다.
> 
> 8. **법적 조치 및 대응**:
>    - 폭력 발생과 관련한 법적 조치, 대응, 보상 절차를 마련합니다.
> 
> 9. **직원 재배치 및 업무 변경**:
>    - 폭력 사고 관련 직원 재배치 및 업무 변경에 대한 조정을 실시합니다.
> 
> 10. **중재 및 해결 기술**:
>     - 가해 이용자 및 보호자와 피해자 간의 중재, 해결 기술을 적용합니다.
> 
> 11. **지속적인 모니터링**:
>     - 가해 이용자 및 피해 사회복지시설 종사자에 대한 지속적인 모니터링을 실시합니다.
> 
> 12. **대피 시 행동요령**:
>     - 대피 시 엘리베이터 사용을 피하고, 부상자 발생 시 응급처치를 실시하며, 대피 후 인원 확인 및 점검을 실시합니다. 2차 재난 발생에 따른 예방조치도 사전에 실시합니다.
> 
> 이와 같은 절차를 통해 사회복지시설에서 발생한 사고에 대해 체계적이고 신속하게 대처할 수 있습니다.

In [ ]:
class chaingpt:
    def __init__(self,api_key,retriever, sys_prompt="",model="gpt-4o"):
        self.template = sys_prompt + '''Answer the question based only on the following context:
        {context}

        Question: {question}
        '''
        self.prompt = ChatPromptTemplate.from_template(self.template)
        self.model = ChatOpenAI(api_key=api_key,model=model, temperature=1)
        self.chainmodel = (
        {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
        | self.prompt  # 프롬프트 템플릿을 적용합니다.
        | self.model  # 모델을 호출합니다.
        | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
        )
    def invoke(self,input_message):
        return self.chainmodel.invoke(input_message)
    
#ex
api_key = OPENAI_API_KEY
retriever = vectordb.as_retriever()
sys_prompt = """사용자의 외로움을 판단하고, 사용자에게 적절한 대화 상대가 되어주기 위한 프롬프트를 출력해주세요. """
gpt = chaingpt(api_key,retriever,sys_prompt)
input_message =  """사용자의 외로움은 뭔가요? 적절한 대화상대가 되어주세요."""
print(gpt.invoke(input_message))
        